<a href="https://colab.research.google.com/gist/SeanHwangG/90c8434a8f53cdac9561cdd6624a101e/untitled2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [16]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torch.autograd import Variable


input_size, hidden_size, batch_size = 6, 6, 1
num_layers = 1

idx2char = ['h', 'i', 'e', 'l', 'o', ' '] 
x_data = [0, 1, 0, 2, 3, 3, 0] # h i h e l l h
y_data = [1, 5, 0, 2, 3, 3, 4] # i   h e l l o
inputs = nn.functional.one_hot(torch.LongTensor(x_data).view(-1,1), input_size).float()
labels = Variable(torch.LongTensor(y_data))

class Model(nn.Module): 
    def __init__(self): 
        super(Model, self).__init__() 
        self.rnn = nn.RNN(input_size = input_size, hidden_size = hidden_size, batch_first = True) 
        
    def forward(self, x, hidden): 
        x = x.view(batch_size, len(x_data), input_size) 
        out, hidden = self.rnn(x, hidden) 
        out = out.view(-1, len(idx2char))
        return hidden, out 
    
    def init_hidden(self): 
        return Variable(torch.zeros(num_layers, batch_size, hidden_size))

model = Model()
print(model)
criterion = torch.nn.CrossEntropyLoss() 
optimizer = torch.optim.Adam(model.parameters(), lr = 0.1) 
hidden = model.init_hidden()

Model(
  (rnn): RNN(6, 6, batch_first=True)
)


In [17]:
for epoch in range(10): 
    _, outputs = model(inputs, hidden) 
    optimizer.zero_grad()

    loss = criterion(outputs, labels)
    loss.backward() 
    optimizer.step()

    _, idx = outputs.max(1)
    idx = idx.data.numpy()
    
    result_str = [idx2char[c] for c in idx.squeeze()] 
    print(f"epoch: {epoch + 1}, loss: {loss.item():.3f} \t {''.join(result_str)}")

epoch: 1, loss: 1.874 	 eeeeeei
epoch: 2, loss: 1.576 	 h ellli
epoch: 3, loss: 1.308 	 h hlllo
epoch: 4, loss: 1.094 	 o hello
epoch: 5, loss: 1.024 	 o hello
epoch: 6, loss: 0.959 	 o hello
epoch: 7, loss: 0.888 	 h hello
epoch: 8, loss: 0.826 	 i hello
epoch: 9, loss: 0.781 	 i hello
epoch: 10, loss: 0.744 	 i hello
